# Text Analysis Group Assignment
    
    AJITH SANIKOMMU       PGID: 12120006
    Anjana Rajan	      PGID: 12120080
    Bhargavi Peddapati	  PGID: 12120067
    Rohini Singh	      PGID: 12120059
    Shantanu Srivastava	  PGID: 12120061

# Importing Libraries

In [1]:
#!pip install vaderSentiment
#!pip install wordcloud
#!pip install -U gensim
#!pip install xgboost
#!pip install plotly
#!pip install textblob
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('omw-1.4')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#!pip install seaborn
import time
t0 = time.time()
import pandas as pd
import numpy as np
import re
import string
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from wordcloud import WordCloud
from pprint import pprint

# nltk
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import seaborn as sns

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import ensemble
import xgboost, string 
import csv,nltk
from sklearn.cluster import KMeans
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from nltk.corpus import webtext
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from gensim.parsing.preprocessing import strip_punctuation, strip_tags, strip_numeric
from nltk.stem.wordnet import WordNetLemmatizer   
from nltk.corpus import stopwords
import string
import seaborn as sns
import calendar

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import metrics

t1 = time.time()
print("Time taken to load/import libraries ", round(t1-t0,3))

Time taken to load/import libraries  3.496


# Loading Data

In [2]:
uber_ride_reviews_df = pd.read_csv("modified_uber_data.csv", encoding= 'unicode_escape')
uber_ride_reviews_df.head(3)

FileNotFoundError: [Errno 2] No such file or directory: 'modified_uber_data.csv'

# Business Problem

<b> To improve the uber services, Uber wants to study customer reviews

# 1. Pre-processing

## Examine the dataset

In [ ]:
uber_ride_reviews_df.head(3)

In [ ]:
uber_ride_reviews_df.shape

In [ ]:
uber_ride_reviews_df.dtypes

In [ ]:
uber_ride_reviews_df.columns

In [ ]:
uber_ride_reviews_df['Rating'].value_counts()

In [ ]:
uber_ride_reviews_df.describe()

## Identifing the columns of interest

Here 'Title', 'Review', 'Rating', 'Date' columns are interested for analysing the data.

## Cleaning of data

### Checking nulls

In [ ]:
uber_ride_reviews_df.isnull().sum()

### Drop Special Characters

In [ ]:
def remove_special_characters(dataCol):
    r = re.compile(r'[^a-zA-Z !@#$%&*_+-=|\:";<>,./()[\]{}\']')
    return r.sub('', dataCol)

uber_ride_reviews_df['Review'] = uber_ride_reviews_df['Review'].apply(remove_special_characters)
uber_ride_reviews_df

### Cleanup html junk

In [ ]:
def remove_html_junk(dataCol):
    text = BeautifulSoup(dataCol, 'html.parser').getText()
    return text

uber_ride_reviews_df['Review'] = uber_ride_reviews_df['Review'].apply(remove_html_junk)
uber_ride_reviews_df

### Trim Data

In [ ]:
def trim_string(dataCol):
    return dataCol.strip()

uber_ride_reviews_df['Review'] = uber_ride_reviews_df['Review'].apply(trim_string)
uber_ride_reviews_df

### Remove certain patterns observed in the text

In [ ]:
def obs_pattern(dataCol):
    r = re.compile(r'^[\s!\s]+$')
    return r.sub('', dataCol)

uber_ride_reviews_df['Review'] = uber_ride_reviews_df['Review'].apply(obs_pattern)
uber_ride_reviews_df

# 2. Basic text and sentiment Analysis

## Basic Text Analysis

<b>Load nltk's English stopswords</b>

In [ ]:
stopwords = nltk.corpus.stopwords.words("english")

### Tokenization

<b>Below are the basic tokenization steps
- <b>Tokenize sentence into words
- <b>Convert words to lowercase
- <b>Consider only Alphanumeric words and exclude other's like numeric etc.
- <b>Remove stopwords

In [ ]:
def tokenize_only(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if(not token in stopwords):            
            if re.search('[a-zA-Z]',token):
                filtered_tokens.append(token)
    return filtered_tokens

In [ ]:
totalvocab_tokenized =[]
for i in uber_ride_reviews_df['Review']:    
    allword_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allword_tokenized) 
print("Get 10 words from list ",totalvocab_tokenized[:10])
print("Total number of tokenized list ", len(totalvocab_tokenized))

### Tokenization With Stemming

<b>Load nltk's stemmer

In [ ]:
stemmer = SnowballStemmer("english")

<b>Below are the basic tokenization steps
- <b>Call tokenize_only method to get list of tokens
- <b>Get stem of each word
- <b>Get final list of Bag of Words

In [ ]:
def tokenize_and_stem(text):    
    filtered_tokens = tokenize_only(text)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [ ]:
totalvocab_stemmed = []
for i in uber_ride_reviews_df['Review']:
    allwords_stemmed = tokenize_and_stem(i)
    totalvocab_stemmed.extend(allwords_stemmed)
totalvocab_stemmed
print("After stemming : ")
print("Get 10 words from list ",totalvocab_stemmed[:10])
print("Total number of tokenized list ", len(totalvocab_stemmed))

### Parts Of Speech 

- <b>On Review column apply tokenization, Removal of stopword, POS

In [ ]:
pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
def token_stop_pos(text):
    tags = pos_tag(tokenize_only(text))
    newlist = []
    for word, tag in tags:
        newlist.append(tuple([word, pos_dict.get(tag[0])]))           
    return newlist

uber_ride_reviews_df['POS tagged'] = uber_ride_reviews_df['Review'].apply(token_stop_pos)
uber_ride_reviews_df.head()

### Lemmatization

- <b>Apply Lemmatization on the POS data

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatize(pos_data):
    lemma_rew = " "
    for word, pos in pos_data:
        if not pos:
            lemma = word
            lemma_rew = lemma_rew + " " + lemma
        else:
            lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
            lemma_rew = lemma_rew + " " + lemma
    return lemma_rew

uber_ride_reviews_df['Lemma'] = uber_ride_reviews_df['POS tagged'].apply(lemmatize)
uber_ride_reviews_df.head()

## Sentiment Analysis with VADER

- <b>Defined a function to calculate Sentiment polarity score using vader
- <b>Create new column 'Vader Sentiment' apply on Lemma Column

In [ ]:
analyzer = SentimentIntensityAnalyzer()
def vadersentimentanalysis(review):
    vs = analyzer.polarity_scores(review)
    return vs['compound']
uber_ride_reviews_df['Vader Sentiment'] = uber_ride_reviews_df['Lemma'].apply(vadersentimentanalysis)

- <b>Defined a function for Labeling Analysis

In [ ]:
def vader_analysis(compound):
    if compound > 0:
        return 'Positive'
    elif compound == 0:
        return 'Neutral'
    else:
        return 'Negative'
uber_ride_reviews_df['Vader Analysis'] = uber_ride_reviews_df['Vader Sentiment'].apply(vader_analysis)
uber_ride_reviews_df.head()

In [ ]:
#pd.set_option('display.max_colwidth',500)
rslt_df = uber_ride_reviews_df.loc[uber_ride_reviews_df['Vader Analysis'] == 'Negative'] 
rslt_df[['Review','Vader Analysis']].head(1)

In [ ]:
rslt_df = uber_ride_reviews_df.loc[uber_ride_reviews_df['Vader Analysis'] == 'Neutral'] 
rslt_df[['Review','Vader Analysis']].head(1)

In [ ]:
rslt_df = uber_ride_reviews_df.loc[uber_ride_reviews_df['Vader Analysis'] == 'Positive'] 
rslt_df[['Review','Vader Analysis']].head(1)

In [ ]:
sentiment_analysis = uber_ride_reviews_df.groupby(['Vader Analysis']).size().reset_index(name='counts')
sentiment_analysis

#### Sentiment Analysis Conclusion

After adjusting compound value to greater than 0 for negative values,we are more number of negative reviews.

The downside of this analysis is that we are getting higher number of positives as well which in reality is not true

In [ ]:
fig = plt.figure(figsize = (8, 5)) 
# creating the bar plot
plt.bar(sentiment_analysis['Vader Analysis'], sentiment_analysis['counts'], color=['red','gray','green'], width = 0.4) 
plt.xlabel("Sentiment Label")
plt.ylabel("Sentiments Counts")
plt.title("Sentiment Analysis")
plt.show()

### Defining Unit Func

- <b>Defined unit method to process one document
- <b>Test it for first Review record

In [ ]:
def vader_unit_func(doc0):
    sents_list0 = tokenize_only(doc0)
    vs_doc0 = []
    sent_ind = []
    for i in range(len(sents_list0)):
        vs_sent0 = analyzer.polarity_scores(sents_list0[i])
        vs_doc0.append(vs_sent0)
        sent_ind.append(i)
        
    # obtain output as DF    
    doc0_df = pd.DataFrame(vs_doc0)
    doc0_df.insert(0, 'sent_index', sent_ind)  # insert sent index
    doc0_df.insert(doc0_df.shape[1], 'sentence', sents_list0)
    return(doc0_df)

%time doc0_df = vader_unit_func(uber_ride_reviews_df['Review'].iloc[0])
doc0_df.head(3)

### Defining Wrapper-Func

- <b>Deinfe Wrapper function to get unit function to each document

In [ ]:
# define wrapper func
def vader_wrap_func(corpus0):
    
    # use ifinstance() to check & convert input to DF
    if isinstance(corpus0, list):
        corpus0 = pd.DataFrame({'text':corpus0})
    
    # define empty DF to concat unit func output to
    vs_df = pd.DataFrame(columns=['doc_index', 'sent_index', 'neg', 'neu', 'pos', 'compound', 'sentence'])    
    
    # apply unit-func to each doc & loop over all docs
    for i1 in range(len(corpus0)):
        doc0 = str(corpus0.Review.iloc[i1])
        vs_doc_df = vader_unit_func(doc0)  # applying unit-func
        vs_doc_df.insert(0, 'doc_index', i1)  # inserting doc index
        vs_df = pd.concat([vs_df, vs_doc_df], axis=0)
        
    return(vs_df)

# test-drive wrapper func
%time uber_ride_reviews_vs_df = vader_wrap_func(uber_ride_reviews_df)    
uber_ride_reviews_vs_df.head(3)

### Plotting

#### Valence Anlaysis

We are not getting a distinguishable difference between positive and negative Reviews using Vader Analysis

In [ ]:
# Define X and Y variable data
#x = np.array([1, 2, 3, 4])
y = uber_ride_reviews_vs_df.compound
x = pd.Series([x for x in range(len(uber_ride_reviews_vs_df))])
fig = plt.figure(figsize = (19, 8))   
plt.plot(x, y)
plt.xlabel("Sentence num")  # add X-axis label
plt.ylabel("Compound valence score")  # add Y-axis label
plt.title("Valence of Uber Review by sentence")  # add title
plt.show()

## Ngrams - (Bigrams, Trigrams.. etc), COGs, wordclouds, phrases

### Ngrams - (Bigrams, Trigrams.. etc)

- <b>NGrams_Generator method will generate n-grams where is dynamic
- <b>We can generate Ngrams by sending n value as 1,2,3... etc

In [ ]:
def NGrams_Generator(testlist,ngram=1):    
    columns = []
    for w in range(1,ngram+1):       
        columns.append('Word'+str(w))
        
    ngramsdf = pd.DataFrame(columns = columns)  
    for t in testlist:           
        words=tokenize_only(t)
        ngrams=zip(*[words[i:] for i in range(0,ngram)])
        df = pd.DataFrame(ngrams,columns =columns)       
        ngramsdf= ngramsdf.append(df,ignore_index = True)       
    return ngramsdf

#### Bigrams

In [ ]:
bigrams_df = NGrams_Generator(uber_ride_reviews_df['Review'],2)
print("Bgram")
bigrams_df.head(5)

#### Bigram Analysis 
Customer Service, Uber Use, Uber Eats, Uber Pass are the most occuring features we have identifed using Bigrams

In [ ]:
bi_count_df = bigrams_df.groupby(['Word1','Word2']).size().reset_index(name='counts').sort_values(['counts'], ascending=False)
bi_count_df.head(5)

#### Trigrams

In [ ]:
trigrams_df = NGrams_Generator(uber_ride_reviews_df['Review'],3)
print("Trigram")
trigrams_df.head(5)

#### Trigram Analysis 
mostly money, timing and charging related issues can be identified from Trigram Analysis

In [ ]:
trigrams_df = trigrams_df.groupby(['Word1','Word2','Word3']).size().reset_index(name='counts').sort_values(
    ['counts'], ascending=False)
trigrams_df.head(5)

# 3. Feature construction and extraction

### Construct DTM

- <b>Build a panda DF, index as stemmed vocabulary and column as tokenized words
- <b>Build DTM under TFIDF scheme

- <b>1. Build/Construct vectorizer by providing Lemmatized Text, n value to calculate Ngram 
- <b>2. Transform vectorizer to get BOW, feature names
- <b>3. Construct dataframe of bag of words
- <b>4. Build TFIDF matrix from bag of words
- <b>5. Get Words of highest weights
- <b>6. Build dataframe with word,count,weight
- <b>7. Get Cosine similarity
- <b>8. Get the Distance matrix

In [ ]:
def buildVector(text,ngram):
    cvector = CountVectorizer(max_df=1.0,  min_df=0.0, vocabulary=None, ngram_range=(1,ngram))
    cvectorfit = cvector.fit(text)
    
    bag_of_words = cvectorfit.transform(text)
    feature_names = cvectorfit.get_feature_names()
    bag_of_words_df = pd.DataFrame(bag_of_words.todense(), columns=feature_names)
    
    tfidftransformer = TfidfTransformer()
    tfidf = tfidftransformer.fit_transform(bag_of_words)
    
    word_cnts = np.asarray(bag_of_words.sum(axis=0)).ravel().tolist()
    df_cnts = pd.DataFrame({'word': feature_names, 'count': word_cnts})
    df_cnts = df_cnts.sort_values('count', ascending=False)
    weights = np.asarray(tfidf.mean(axis=0)).ravel().tolist()
    df_weights = pd.DataFrame({'word': feature_names, 'weight': weights})
    df_weights = df_weights.sort_values('weight', ascending=False)

    df_weights = df_weights.merge(df_cnts, on='word', how='left')
    df_weights = df_weights[['word', 'count', 'weight']]

    cos_sim = cosine_similarity(tfidf, tfidf)
    samp_dist = 1 - cos_sim
    
    return cvectorfit, feature_names, bag_of_words_df, tfidf, df_weights, cos_sim, samp_dist

In [ ]:
cvec, feature_names, bag_of_words_df, tfidf, df_weights, cos_sim, samp_dist = buildVector(uber_ride_reviews_df['Lemma'],2)
df_tfidf = pd.DataFrame(tfidf.todense(), columns=feature_names)

In [ ]:
print("Features:")
feature_names[:10]

In [ ]:
print("Weights:")
df_weights.head(5)

In [ ]:
sentences = uber_ride_reviews_df['Lemma'].values.tolist()
print("cos_sim[%d,%d] (a square matrix of length and width ) " % (len(sentences), len(sentences)))
df = pd.DataFrame(np.array(cos_sim))
df.head(3)

In [ ]:
print("df_bag_of_words[%d,%d]:" % (len(sentences), len(feature_names)))
bag_of_words_df.head(3)

In [ ]:
s_word_freq = pd.Series(df_weights['count'])
s_word_freq.index = df_weights['word']
di_word_freq = s_word_freq.to_dict()

### Wordclouds

In [ ]:
def generate_word_cloud(text):
    wordcloud = WordCloud(
        width = 3000,
        height = 2000,
        background_color = 'white').generate_from_frequencies(text)
    fig = plt.figure(
        figsize = (15, 8),
        facecolor = 'k',
        edgecolor = 'k')
    plt.imshow(wordcloud, interpolation = 'bilinear')
    plt.axis('off')
    plt.tight_layout(pad=0)
    plt.show()

#### Word Cloud Analysis

The most prominent features we can identify are:

1. Uber
2. Driver
3. Ride
4. App
5. Charge

Since, 90% of the ratings are negative, we can say that the features mentioned above are very likely to be Issue related or Negative

###### Recommendations - Driver, Ride, App & Charging related issues must be addressed as these words frequency is the highest



In [ ]:
generate_word_cloud(di_word_freq)

### PCA - Dimensionality Reduction 

In [ ]:
# Dimensionality reduction using PCA, reduce the tfidf matrix to just 2 features
X = tfidf.todense()
pca = PCA(n_components=2)
pca.fit(X)
X_pca = pca.transform(X)

print("X_pca now has just 2 columns:")
print(X_pca)

### K-means clusters

In [ ]:
n_clusters = np.unique(uber_ride_reviews_df['Title']).shape[0]
print("n_clusters:", n_clusters)

km_model = KMeans(n_clusters=n_clusters, max_iter=10, n_init=2, random_state=0)

# K-means (from number of features in input matrix to n_clusters)
km_model.fit(X_pca)
df_centers = pd.DataFrame(km_model.cluster_centers_, columns=['x', 'y'])

plt.figure(figsize=(15,8))
plt.suptitle('PCA features colored by class; grey circles show the k-means centers')
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=km_model.labels_, s=50, cmap='jet')
plt.scatter(df_centers['x'], df_centers['y'], c='grey', s=500, alpha=0.2);

dy = 0.04
for i, txt in enumerate(km_model.labels_):
    plt.annotate(txt, (X_pca[i, 0], X_pca[i, 1] + dy))

In [ ]:
print("km_model.labels_:", km_model.labels_)
print("This corresponds to the sentence labels shown below as follows:")
print(uber_ride_reviews_df['Title'].tolist())
print("---")
print("df_centers:")
print(df_centers)

### Implementation of LDA using Sklearn

In [ ]:
lda_model = LatentDirichletAllocation(n_components=6,max_iter=20,random_state=20)

In [ ]:
xtopic = lda_model.fit_transform(tfidf)
xtopic

In [ ]:
topic_words =lda_model.components_
beta_df = pd.DataFrame(topic_words)
topicNames=['topic' + format(x+1, '02d') for x in range(len(beta_df))]
topicNames_series = pd.Series(topicNames)
final = beta_df.iloc[:8, :8].T
final.rename(columns=topicNames_series, inplace=True)
final

### Compute Fit metrics for topic models

In [ ]:
n_top_words=5
for i,topic_dist in enumerate(topic_words):
    sorted_topic_dist = np.argsort(topic_dist)
    topic_words = np.array(feature_names)[sorted_topic_dist]
    topic_words = topic_words[:n_top_words:-1]
    print("Topic",str(i+1),topic_words)

#### LDA Analysis

The topics we have identified from LDA are following:

Topic 1 ['driver' 'charge' 'uber' ... 'app informs' 'stop service' 'go work']
Topic 2 ['app' 'uber' 'service' ... 'plane' 'informs' 'app informs']
Topic 3 ['ride' 'uber' 'get' ... 'informs' 'app informs' 'stop service']
Topic 4 ['uber' 'ride' 'get' ... 'reserve time' 'perfect' 'training']
Topic 5 ['uber' 'use' 'driver' ... 'informs' 'stop service' 'gig']
Topic 6 ['uber' 'driver' 'get' ... 'app informs' 'informs' 'stop service']

The drawback of LDA is that we do not get the weights of each keyword associated with each Topic. Therefore, to differentiate a topic is difficult in LDA

### Annotating the topics the documents

In [ ]:
doc_topic = lda_model.transform(tfidf)
for n in range(doc_topic.shape[0]):
    topic_doc =doc_topic[n].argmax()
    print("Document",n+1,"Topic ", topic_doc)

### Implementation of LDA using Gensim

We are using Gensim Topic modelling over LDA for running classification because LDA has limitations such as
1. Fixed K - we need to know the number of topics beforehand
2. Uncorrelated Topics - correlations are not captured using LDA

In Gensim model, we are trying to explore as many features as the model can identify using coherence values and perplexity fitb

#### Step 1: Text cleaning and Tokenisation

Clean the text and then remove any additional stop words
NLTK extended Stop words list
Below we are removing common words that do not provide latent feature specific words. 
Words like uber, driver, couldnt, taking, dude, something etc

### Please note that we are running Gensim Model and Text Classification for Reviews with Ratings 1 or 2

We want to identify the features that are contributing to Bad Ratings

In [ ]:
from nltk.corpus import stopwords
uber_ride_reviews_df['Review']=uber_ride_reviews_df['Review'].str.replace('driver', '')
uber_ride_reviews_df = uber_ride_reviews_df[(uber_ride_reviews_df.Rating==1)|(uber_ride_reviews_df.Rating==2)]
# NLTK Stop words extended
stop_words = stopwords.words('english')
stop_words.extend(['uber','away', 'every','certain','finally','got','get', 'im','even','cant', 'would',
                  'said', 'could', 'wanted', 'needed', 'end', 'taking', 'dude','gotten', 'since', 'le', 'thus',
                  'tx', 'whatever', 'therefore','ever', 'pa', 'wasnt','still', 'gave','yall','something', 'theyre',
                  'dont', 'need',  'wont', 'doesnt',  'nothing', 'ive','saying','u','please'])

In [ ]:
lemma = WordNetLemmatizer()
def textClean(text0):
    text1 = [strip_punctuation(doc) for doc in text0]
    text1 = [strip_tags(doc) for doc in text1]
    text1 = [strip_numeric(doc) for doc in text1]
    text1 = [[" ".join([i for i in doc.lower().split() if i not in stop_words])] for doc in text1]
    text2 = [[word for word in ' '.join(doc).split()] for doc in text1]
    normalized = [[" ".join([lemma.lemmatize(word) for word in ' '.join(doc).split()])] for doc in text1]
    return normalized

corpus1 = textClean(uber_ride_reviews_df['Review'])

#### Step 2: Creating corpus for Topic Modelling

Gensim supports building of dictionary object which we are applying on n-gram we have built.

After, creating the dictionary, we are creating the TF or term frequency count based on DTM 

We have kept the min_count and threshold of these values on a bit higher side to get better bigrams from the Reviews as the reviews are quite descriptive

In [ ]:
## DTM building etc via gensim
corpus2 = [[word for word in ' '.join(doc).split()] for doc in corpus1]  # word_tokenize first

# Building the bigram and trigram models
bigram = gensim.models.Phrases(corpus2, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[corpus2], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# print(trigram_mod[bigram_mod[corpus2[0]]])  # See trigram example

id2word = corpora.Dictionary(corpus2)  # Create Dictionary
corpus = [id2word.doc2bow(text) for text in corpus2]  # Building gensim corpus. TF DTM creation.
print(corpus[:1])  # View one doc in abstract form

In [ ]:
# Human readable format of corpus (term-frequency)
a0 = [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]
[x for (x, y) in a0[0] if y >4]

#### Step 3: Building the Topic Model

we have given the number of topics as 4 to extract latent topics

In [ ]:
# Build LDA model for (say) K=4 topics
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

#### Step 4: Obtain Factor Matrices

We obtain two factor Matrices from Gensim data
1. beta - maps tokens to topics
2. gamma - maps docs to topics

In [ ]:
## def func for beta_df
import pandas as pd

## obtain the factor matrices - beta
def build_beta_df(lda_model=lda_model, id2word=id2word):
    beta = lda_model.get_topics()  # shape (num_topics, vocabulary_size).
    beta_df = pd.DataFrame(data=beta)

    # convert colnames in beta_df 2 tokens
    token2col = list(id2word.token2id)
    beta_df.columns = token2col
    # beta_df.loc[0,:].sum()  # checking if rows sum to 1

    # convert rownames too, eh? Using format(), .shape[] and range()
    rowNames=['topic' + format(x+1, '02d') for x in range(beta_df.shape[0])]
    rowNames_series = pd.Series(rowNames)
    beta_df.rename(index=rowNames_series, inplace=True)
    return(beta_df)

# invoke func
beta_df = build_beta_df(lda_model=lda_model, id2word=id2word)
beta_df.iloc[:8, :8]

In [ ]:
corpus0 = uber_ride_reviews_df['Review'].to_list()
# func to get gamma matrix by looping using list.comp
def build_gamma_df(lda_model, corpus0):
    gamma_doc = []  # empty list 2 populate with gamma colms
    num_topics = lda_model.get_topics().shape[0]
    
    for doc in range(len(corpus0)):
        doc1 = corpus0[doc].split()
        bow_doc = id2word.doc2bow(doc1)
        gamma_doc0 = [0]*num_topics  # define list of zeroes num_topics long
        gamma_doc1 = lda_model.get_document_topics(bow_doc)
        gamma_doc2_x = [x for (x,y) in gamma_doc1]#; gamma_doc2_x
        gamma_doc2_y = [y for (x,y) in gamma_doc1]#; gamma_doc2_y
        for i in range(len(gamma_doc1)):
            x = gamma_doc2_x[i]
            y = gamma_doc2_y[i]
            gamma_doc0[x] = y  # wasn't geting this in list comprehension somehow 
        gamma_doc.append(gamma_doc0)
        
    gamma_df = pd.DataFrame(data=gamma_doc)  # shape=num_docs x num_topics
    topicNames=['topic' + format(x+1, '02d') for x in range(num_topics)]
    topicNames_series = pd.Series(topicNames)
    gamma_df.rename(columns=topicNames_series, inplace=True)
    return(gamma_df)

# now apply func
gamma_df = build_gamma_df(lda_model=lda_model, corpus0=corpus0)
gamma_df.iloc[:8, :]

In [ ]:
gamma_df.iloc[2,:].sum() 

#### Step 6: Compute fit metrics for Topic Models

We iterate the model with varying num_topics to obtain optimal number of topics. 

Perplexity score tells us the model fitness. 

By running this model on different num_topics, the final perplexity score we are achieving is below

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # model fit metric. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=corpus, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
## compute coherence score (akin to LMD?)
def compute_coherence_values(dictionary, corpus, texts, limit, start, step):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    num_topics1 = [i for i in range(start, limit, step)]
    for num_topics in num_topics1:
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics, random_state=100,
                                           update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values  # note, list of 2 objs returned

In [ ]:
# Can take a long time to run.
start1=2
limit1=19
step1=1

model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, 
                                                        texts=corpus2, start=start1, limit=limit1, step=step1)

print(coherence_values)

In [ ]:
# obtain optimal topic number
coher = list(enumerate(coherence_values))  # create an index for each list elem
index_max = [x for (x,y) in coher if y==max(coherence_values)]  # obtain index num corres to max coherence value
Optimal_numTopics = int(str(index_max[0]))+2  # convert that list elem into integer (int()) via string (str())
print(Optimal_numTopics) 

In [ ]:
## Plot the change in coherence score with num_topics
import matplotlib.pyplot as plt
%matplotlib inline

start1=2
limit1=19
step1=1
# Show graph
x = range(start1, limit1, step1)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.axvline(x=Optimal_numTopics, color='r')
plt.show()

In [ ]:
len(model_list)

#### Interpreting optimal topics

We have built the LDA model with 10 different topics in which multiple keywords contribute to a certain weightage(importance) for that topic

In [ ]:
# seems optimal num_topics is 10
optimal_model = model_list[Optimal_numTopics]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=17))

#### Step7: Exploring topics obtained from optimal model

We are identifying the dominant topic and then mapping it back with the main document

In [ ]:
# Get main topic in each document
gamma_df = build_gamma_df(lda_model=optimal_model, corpus0=corpus0)
#gamma_df.iloc[:8,:8]

row0 = gamma_df.values.tolist()
row=[]
for i in range(len(row0)):
    row1 = list(enumerate(row0[i]))
    row1_y = [y for (x,y) in row1]
    max_propn = sorted(row1_y, reverse=True)[0]
    row2 = [(i, x, y) for (x, y) in row1 if y==max_propn]
    row.append(row2)

row[0:7]

In [ ]:
sent_topics_df = pd.DataFrame()
for row1 in row:
    for (doc_num, topic_num, prop_topic) in row1:
        wp = optimal_model.show_topic(topic_num)
        topic_keywords = ", ".join([word for word, prop in wp])
        sent_topics_df = sent_topics_df.append(pd.Series([int(doc_num), int(topic_num), 
                                                          round(prop_topic,4), 
                                                          topic_keywords]), 
                                                       ignore_index=True)
    
sent_topics_df.columns = ['Doc_num', 'Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']    
sent_topics_df.iloc[:8, :]

In [ ]:
# Add original text to the end of the output
contents = pd.Series(corpus0)
sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
#return(sent_topics_df)
sent_topics_df.columns = ['Doc_num', 'Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords', 'contents']
sent_topics_df.iloc[:8,:]

In [ ]:
test = sent_topics_df['Dominant_Topic'].value_counts()
test

#### Conclusion of Gensim Topic Model

We have identified 10 dominant topics with following interpretation

#Topic Descriptors sorted by size or count of reviews
 3 --> Service (related issues)
 2 --> Ride (related issues)
 1 --> App (related issues)
 11 --> Payment (related issues)
 10 --> Advertising (related issues)
 6 --> Family_Account (related issues)
 4 --> Pricing (related issues)
 5 --> Vehicle (related issues)
 7 --> Experience (related issues)
 9 --> Policy (related issues)
 0 --> Availability (related issues)
 8 --> Reliability (related issues)

# 4. Text Classification

#### Step 1 : Utilising topics obtained from Gensim model for Text Classification 

We are utilising the topics we have obtained from Gensim for Text classification of Reviews 

In [ ]:
data_subset = sent_topics_df[['contents','Dominant_Topic']].dropna(how = 'any')
print(data_subset.shape)
data_subset['Dominant_Topic'] = data_subset['Dominant_Topic'].astype(int)
data_subset.head()

In [ ]:
#Naming the Dominant Topics
#Topic Descriptors sorted by size or count of reviews
# 3 --> Service (related issues)
# 2 --> Ride (related issues)
# 1 --> App (related issues)
# 11 --> Payment (related issues)
# 10 --> Advertising (related issues)
# 6 --> Family_Account (related issues)
# 4 --> Pricing (related issues)
# 5 --> Vehicle (related issues)
# 7 --> Experience (related issues)
# 9 --> Policy (related issues)
# 0 --> Availability (related issues)
# 8 --> Reliability (related issues)

data_subset['Topic_desc'] = np.where(data_subset['Dominant_Topic']==0, 'Avalibility',
                   np.where(data_subset['Dominant_Topic']==1, 'App',
                   np.where(data_subset['Dominant_Topic']==2, 'Ride', 
                   np.where(data_subset['Dominant_Topic']==3, 'Service',
                   np.where(data_subset['Dominant_Topic']==4, 'Pricing',
                   np.where(data_subset['Dominant_Topic']==5, 'Vehicle',
                   np.where(data_subset['Dominant_Topic']==6, 'Account',
                   np.where(data_subset['Dominant_Topic']==7, 'Experience',
                   np.where(data_subset['Dominant_Topic']==8, 'Reliability',
                   np.where(data_subset['Dominant_Topic']==9, 'Policy',
                   np.where(data_subset['Dominant_Topic']==10, 'Advertising','Payment')))))))))))
data_subset

In [ ]:
# ensure records are unique
categories = list(data_subset['Topic_desc'].drop_duplicates())
categories

#### Step2: Feature Extraction

We are using Bag of Words to compute tf and tf-idf

In [ ]:
#Feature extraction
# We would be filtering the words which occur in less than <2% of the documents and >98% of the documents 
# since these dont add much value to our analysis

tf = CountVectorizer(min_df = 0.02 , max_df= 0.98, stop_words='english')
tfidf = TfidfVectorizer(min_df = 0.02 , max_df= 0.98, stop_words='english')

%time dtm = pd.DataFrame(tf.fit_transform(data_subset.iloc[:,0]).toarray(),columns=tf.get_feature_names()) # 12.3s
%time dtm_idf = pd.DataFrame(tfidf.fit_transform(data_subset.iloc[:,0]).toarray(),columns=tfidf.get_feature_names()) # 12.5s

In [ ]:
dtm.head() # what does the DTM look like?

In [ ]:
dtm_idf.head()

In [ ]:
data_subset.iloc[:,-2]

#### Step 3: Training the model

Firstly, we need to split the data into test and train

In [ ]:
#Training the model
X_train, X_test, y_train, y_test = train_test_split(dtm,data_subset.iloc[:,-2])
X_train_idf, X_test_idf, y_train_idf, y_test_idf = train_test_split(dtm_idf,data_subset.iloc[:,-2])

#### Naive Bayes Model Interpretation:

The model is 51.8% accurate in predicting the dominant topic obtained from Gensim using the DTM.

In [ ]:
#Naive Bayes
# Train the classifier on training data set
clf = MultinomialNB()
%time clf.fit(X_train,y_train) # 6.3s

# Prediction accuracy on test dataset
y_pred = clf.predict(X_test)
clf.score(X_test,y_test)

#### TF IDF Matrix Interpretation:

The model is 49.1% accurate in predicting the topic using TF-IDF Matrix

In [ ]:
#TF_IDF MATRIX

# Train the classifier on idf data set
%time clf.fit(X_train_idf,y_train_idf) # 0.2s
y_pref_idf = clf.predict(X_test_idf)

# Prediction accuracy
clf.score(X_test_idf,y_test_idf)

#### Naive Bayes Model Heat Map and Interpretation

1. Naive Bayes Model has better Accuracy than TF-IDF Matrix
2. Ride is getting confused with Policy, Experience, Availability and Payment


In [ ]:
# Compute the confusion matrix and display it as a heatmap.
conf_mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(15,10))

# neat heat map
sns.heatmap(conf_mat , cmap="RdBu_r", xticklabels = categories, yticklabels = categories)
plt.ylabel('True values')
plt.xlabel('Predicted values')
plt.show()

#### Random Forest Classifier Interpretation and Mapping

1. The model accuracy is 49.05% for TF and 43.39% for TF_IDF
2. The resuls are same as Naive Bayes model heat map


In [ ]:
#Random Forest Classifier
# Train the classifier on training data set
clf = clf = RandomForestClassifier(n_estimators=50, max_depth=5,random_state=0)
%time clf.fit(X_train,y_train)

# Prediction accuracy on test dataset
y_pred = clf.predict(X_test)
print("Accuracy on TF is : " + str(clf.score(X_test,y_test)))

# Train and test the classifier on idf data set
%time clf.fit(X_train_idf,y_train_idf)
y_pref_idf = clf.predict(X_test_idf)
print("Accuracy on TF-IDF is : " + str(clf.score(X_test_idf,y_test_idf)))

# Compute the confusion matrix and display it as a heatmap.
conf_mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(conf_mat , cmap="Blues", xticklabels = categories, yticklabels = categories)
plt.ylabel('True values')
plt.xlabel('Predicted values')
plt.show()

#### Logistic Regression Interpretation and Mapping

1. The model accuracy is 49.05% for TF and 54.71% for TF_IDF. Therefore,this model is giving us better results
2. Ride is getting confused with Experience, Advertising & Policy
3. In addition to Ride, Advertising & Policy, Experience is also getting confused with App, Pricing and Vehicle

##### Recommendations based on Logistic Regression

1. To improve experience which is very well correlated with sentiment words like, horrible, last, reliability according to Gensim Model, Uber needs to improve the overall experience of the customer
2. Experience is very well confused with Policy, App, Pricing, & Vehicle. So, Uber needs to focus on improving them, to improve the overall experience of the customer
3. Ride is also a prominent feature, which according to Gensim Model has keywords like charge, money etc, therefore we need to improve Experience, Adveritsing and Policy
4. Policy, according to Gensim model, is related to masks, measure, airport and reports, therefore, Policy-wise, ber needs to improve and advertise it's policies in a better way

In [ ]:
# Train the classifier on training data set
clf = LogisticRegression(random_state=0)
%time clf.fit(X_train,y_train)
 
# Prediction accuracy on test dataset
y_pred = clf.predict(X_test)
print("Accuracy on TF is : " + str(clf.score(X_test,y_test)))
 
# Train and test the classifier on idf data set
%time clf.fit(X_train_idf,y_train_idf)
y_pref_idf = clf.predict(X_test_idf)
print("Accuracy on TF-IDF is : " + str(clf.score(X_test_idf,y_test_idf)))
 
# Compute the confusion matrix and display it as a heatmap.
conf_mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(conf_mat , cmap="RdBu_r", xticklabels = categories, yticklabels = categories)
plt.ylabel('True values')
plt.xlabel('Predicted values')
plt.show()

### Data Field Analysis

In [ ]:
Days = uber_ride_reviews_df['Day'].value_counts()
Days.plot(kind='bar', color = 'blue', figsize=(10,5))
plt.xlabel('Day')
plt.ylabel('Frequency')
plt.title('Number of Reviews vs Days')

In [ ]:
#Seasonality
Days = uber_ride_reviews_df['Month'].value_counts()
Days.plot(kind='bar', color = 'blue', figsize=(10,5))
plt.xlabel('Month')
plt.ylabel('Frequency')
plt.title('Number of Reviews vs Month')

In [ ]:
#Year
Days = uber_ride_reviews_df['Year'].value_counts()
Days.plot(kind='bar', color = 'blue', figsize=(10,5))
plt.xlabel('Year')
plt.ylabel('Frequency')
plt.title('Number of Reviews vs Year')